In [1]:
import pandas as pd
import numpy as np

### 1. Načtení souboru, vynechání 1. sloupce

In [2]:
# Načíst data, a vynechat první sloupec:

parkovani = pd.read_csv('parkovani.csv', parse_dates = ['zahajeni_parkovani', 'ukonceni_parkovani'])
parkovani = parkovani.drop(parkovani.columns[0], axis = 1)

#parkovani.head()


### 2. Počet záznamů

In [3]:
celkem_zaznamu = parkovani.shape[0]
print(f'V souboru je celkem {celkem_zaznamu} záznamů.')

V souboru je celkem 62033 záznamů.


In [4]:
# Odstranění řádků, kde není zapsán čas, jinak nevychází správně průměry
parkovani = parkovani.dropna()

In [5]:
celkem_zaznamu = parkovani.shape[0]
print(f'V souboru je celkem {celkem_zaznamu} záznamů, ve kterých nejsou prázdné řádky.')

V souboru je celkem 59699 záznamů, ve kterých nejsou prázdné řádky.


### 3. Přidání sloupce čas, průměrná doba parkování v jednotlivých částech

In [6]:
zacatek = parkovani.zahajeni_parkovani
konec = parkovani.ukonceni_parkovani
delka = konec - zacatek
parkovani.insert(0, 'čas', delka)

parkovani['nove_zony'] = parkovani['parkovaci_zona'].str.split('-').str[0]

parkovani['čas'] = parkovani['čas'].values.astype('int64')

sloupce = ['nove_zony', 'čas']
prumer = parkovani[sloupce].groupby('nove_zony').mean()


In [7]:
prumer['čas'] = prumer['čas'].astype('datetime64[ns]') 
prumer['čas2'] = prumer['čas'].dt.time
for i in prumer.index:
    print(f'Průměrná doba parkování v části {i} je {prumer.čas2[i]}.')

Průměrná doba parkování v části P1 je 02:03:33.831890.
Průměrná doba parkování v části P13 je 01:45:24.081784.
Průměrná doba parkování v části P16 je 02:10:32.442028.
Průměrná doba parkování v části P2 je 04:20:13.011624.
Průměrná doba parkování v části P22 je 02:04:32.167346.
Průměrná doba parkování v části P3 je 02:56:22.286726.
Průměrná doba parkování v části P4 je 05:40:30.361272.
Průměrná doba parkování v části P5 je 05:35:47.258156.
Průměrná doba parkování v části P6 je 04:05:24.969090.
Průměrná doba parkování v části P7 je 02:31:33.922587.
Průměrná doba parkování v části P8 je 04:13:54.198342.


### 4) Kolik se vybralo v jednotlivých částí celkem

In [8]:
filtr = parkovani['kanal'] == 'PARKMACHINE'
sloupce = ['nove_zony', 'cena']
celkem_park = parkovani[filtr][sloupce].groupby('nove_zony').sum()

In [9]:
filtr = parkovani['kanal'] == 'VPH'
sloupce = ['nove_zony', 'cena']
celkem_vph = parkovani[filtr][sloupce].groupby('nove_zony').sum()

In [10]:
vysledek = celkem_vph

In [11]:
vysledek['cena_parkmachine'] = celkem_park['cena']
vysledek['cena_vph'] = celkem_vph['cena']
del vysledek['cena']
#vysledek.rename(index = {'nove_zony': 'parkovaci_zony'})
vysledek = vysledek.reset_index()
vysledek = vysledek.rename(columns = {'nove_zony': 'parkovaci_zony'})

In [12]:
vysledek.to_csv('output/vybrano.csv', index = False)